In [110]:
from pyspark.sql.types import StructType
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import math

In [111]:
my_conf = SparkConf()
my_conf.set("spark.app.name", "w12_Spark_1")
my_conf.set("spark.master", "local[*]")
my_conf.set("spark.driver.extraClassPath", "C:/Users/Vinoth/Downloads/sqljdbc_12.4.2.0_enu/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre8.jar")

spark = SparkSession.builder.config(conf = my_conf).getOrCreate()
spark


In [112]:
database_url = "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true"

database_name = "Learn_Sql"

jdbc_properties = {

"user": "Vinoth",

"password": "1234",

"driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"

}

In [113]:
# df_min_max = (spark.read
# .format("jdbc")
# .option("url", "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true;databaseName=Learn_Sql;")
# .option("query","select min(cost) as min_row, max(cost) as max_row from products")
# .option("user", "Vinoth")
# .option("password", "1234")
# .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
# .load().collect())

# min_value, max_value = df_min_max[0][0], df_min_max[0][1]

In [114]:
# df = (spark.read
#       .option("numPartitions", 3)
#       .option("PartitionColumn", "cost")
#       .option("lowerBound", int(min_value))
#       .option("upperBound", int(max_value))
#       .jdbc(url=f"{database_url};databaseName={database_name};", table="products", properties=jdbc_properties))

In [115]:
tickets_df = (spark.read
      .jdbc(url=f"{database_url};databaseName={database_name};", table="tickets", properties=jdbc_properties))

holidays_df = (spark.read
      .jdbc(url=f"{database_url};databaseName={database_name};", table="holidays", properties=jdbc_properties))

In [116]:
tickets_df = tickets_df.withColumn("no_of_days",abs(datediff("create_date", "resolved_date")))

In [117]:
tickets_df = tickets_df.withColumn('week', 2*floor(datediff('resolved_date','create_date')/7))

In [118]:
tickets_df = tickets_df.withColumn('business_days', expr("no_of_days - week"))

In [119]:
tickets_df.show()

+---------+-----------+-------------+----------+----+-------------+
|ticket_id|create_date|resolved_date|no_of_days|week|business_days|
+---------+-----------+-------------+----------+----+-------------+
|        1| 2022-08-01|   2022-08-03|         2|   0|            2|
|        2| 2022-08-01|   2022-08-12|        11|   2|            9|
|        3| 2022-08-01|   2022-08-16|        15|   4|           11|
+---------+-----------+-------------+----------+----+-------------+



In [120]:
holidays_df.show()

+------------+----------------+
|holiday_date|          reason|
+------------+----------------+
|  2022-08-11|           Rakhi|
|  2022-08-15|Independence day|
+------------+----------------+



In [121]:
join_condition = holidays_df["holiday_date"].between(tickets_df["create_date"],tickets_df["resolved_date"])
join_df = tickets_df.join(holidays_df,join_condition, 'left')
join_df.show()

+---------+-----------+-------------+----------+----+-------------+------------+----------------+
|ticket_id|create_date|resolved_date|no_of_days|week|business_days|holiday_date|          reason|
+---------+-----------+-------------+----------+----+-------------+------------+----------------+
|        1| 2022-08-01|   2022-08-03|         2|   0|            2|        NULL|            NULL|
|        2| 2022-08-01|   2022-08-12|        11|   2|            9|  2022-08-11|           Rakhi|
|        3| 2022-08-01|   2022-08-16|        15|   4|           11|  2022-08-11|           Rakhi|
|        3| 2022-08-01|   2022-08-16|        15|   4|           11|  2022-08-15|Independence day|
+---------+-----------+-------------+----------+----+-------------+------------+----------------+



In [122]:
agg_df = join_df.groupBy("ticket_id", "create_date","resolved_date", "business_days")\
.agg(expr("count(reason) as holiday_cnt"))
agg_df.show()

+---------+-----------+-------------+-------------+-----------+
|ticket_id|create_date|resolved_date|business_days|holiday_cnt|
+---------+-----------+-------------+-------------+-----------+
|        3| 2022-08-01|   2022-08-16|           11|          2|
|        2| 2022-08-01|   2022-08-12|            9|          1|
|        1| 2022-08-01|   2022-08-03|            2|          0|
+---------+-----------+-------------+-------------+-----------+



In [123]:
agg_df.withColumn("actual_business_days", expr("business_days - holiday_cnt")).show()

+---------+-----------+-------------+-------------+-----------+--------------------+
|ticket_id|create_date|resolved_date|business_days|holiday_cnt|actual_business_days|
+---------+-----------+-------------+-------------+-----------+--------------------+
|        3| 2022-08-01|   2022-08-16|           11|          2|                   9|
|        2| 2022-08-01|   2022-08-12|            9|          1|                   8|
|        1| 2022-08-01|   2022-08-03|            2|          0|                   2|
+---------+-----------+-------------+-------------+-----------+--------------------+



In [124]:
spark.stop()